# Comparison of performance with other algorithms

The performance of the deep learning agent is compared with a classical control algorithm (PID control) and a tabular reinforcement learning algorithm (SARSA) in order to determine how deep learning is better in the case of articulated robots with a large state space.


In [ ]:
from ipynb.fs.full.PianoHandv1 import *
# Specify the goal (key) to test here. 
test_key = 'C'

## Classical Control: Proportional-Integral-Derivative (PID) control

In a PID controller, the actual value of the output is compared with the desired output and the current action adjusted based on the error between the two. The desired output signal is calculated by summing the individual responses of the proportional, integral and derivative components of the system.
The following formula is used to calculated the output signal.
$$ Output = K_p*e(t) + K_i*\int_{0}^{t} e(t) \,dt + K_d*\frac{d}{dt}*(e(t)) $$

The following code implements the PID control in our custom environment and assesses its performance. This code has been adapted from https://gist.github.com/HenryJia/23db12d61546054aa43f8dc587d9dc2c.

In [ ]:
import numpy as np
import gym

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

env = PianoHandEnv(test_key)
desired_state = np.array([45, 45, 45, 45, 45, 45, 55, 45])
desired_mask = np.array([1, 1, 1, 1, 1, 1, 1, 1])
action_distribution = np.zeros(15)
total_episodes = 100

# The parameters have been tuned to give us the best result
P, I, D = 0.7, 0.003, 0.5
inte = []
der = []

reward_count_pid = []

for episode in range(total_episodes):
    
    st = env.reset()
    state = (st[0][0], st[0][1], st[1][0], st[1][1], st[2][0], st[2][1], st[3][0], st[3][1])
    prev_error = 0
         
    for t in range(500):
        error = state - desired_state
        integral = 0
        derivative = 0
       
        # Calculating the output using the PID control formula
        integral += error
        derivative = error - prev_error
        prev_error = error
        inte.append(integral)
        der.append(derivative)

        pid = np.dot(P * error + I * integral + D * derivative, desired_mask)
        action = sigmoid(pid)
        
        # Since a sigmoid function is used, the actions need to be normalized to the range [0,15]
        action = action * 15
        action = np.round(action).astype(np.int32)
        
        action_distribution[action]= action_distribution[action] + 1
        st, reward, done, info, final = env.step(action)
        reward_count_pid.append(reward)
        state = (st[0][0], st[0][1], st[1][0], st[1][1], st[2][0], st[2][1], st[3][0], st[3][1])
        if done:
            break
        
    print("Episode {} Average rewards = {}". format(episode+1, np.mean(reward_count_pid)), end = '\r')
    time.sleep(0.25)

## Results obtained from PID control
The following graphs display the moving average of rewards earned over all episodes and the action distrubution curve which shows the frequency of choosing an action using this algorithm.



In [ ]:
def moving_average_rewards(rewards, num_of_episodes, partition_parameter):
    rewards_per_partition = np.split(np.array(rewards), num_of_episodes/partition_parameter)
    average_rewards_per_partition=[]
    episode_number=[]
    counter=0
    
    for r in rewards_per_partition: #calculating the average reward for each partition
        average_rewards_per_partition.append(sum(r/partition_parameter))
        episode_number.append(counter)
        counter+= partition_parameter
        
    #Design of the plot
    plt.figure()
    plt.rcParams["font.family"] = "serif"   
    plt.plot(episode_number, average_rewards_per_partition, color='black')
    plt.ylabel("Moving average of total reward")
    plt.xlabel("Episode")
    plt.title("Moving Average of Total Reward per Episode")
    plt.grid('on')

moving_average_rewards(reward_count_pid, total_episodes, 1)

In [ ]:
# Plot the action distribution
plt.plot(action_distribution, color = 'black')
plt.grid('on')
plt.xlabel("Actions")
plt.ylabel("Number of Times Action was Executed")
plt.title("Action Distribution")

## Tabular Reinforcement learning: SARSA 

SARSA is a model free method, on-policy method of approximating the action value function to obtain the optimal policy, using the Temporal Difference (TD) Methods. It is a tabular method where the Q-values are stored in a look-up table, which is used to pick the best action at any given state. The Q-values are calculated using the following equation:
$$Q(S_{t},A_{t})\leftarrow Q(S_{t},A_{t}) + \alpha*[R_{t+1} + \gamma*Q(S_{t+1},A_{t+1}) - Q(S_{t},A_{t})]$$

Since our state space is quite large ($31^8$ states), this tabular method is not feasible for training the model. In this code, we have demonstrated the use of SARSA to train only one finger of the robotic hand, which reduces the state space to $31^2$ states. 

In [ ]:
# Setting parameters
alpha             = 0.4           # Learning rate - rate at which Q values are updated
gamma             = 0.9           # Discount rate - how important are the immediate rewards vs later rewards
epsilon_normal    = 0.01          # Used to balance exploring versus exploiting
total_episodes    = 100           # Total episodes for training

# For decaying epsilon greedy
decay             = True          # Change to True for implementing decay
epsilon_decay     = 0.95          # Initial epsilon value
min_epsilon       = 0.01          # Minimum value to decay until  
decay_rate        = 0.01          # Decay rate

def e_greedy(state, Q, epsilon):
    # If the random number generated is smaller than epsilon, choose to explore, otherwise take best action.
    if np.random.uniform(0,1) < epsilon:          
        action = random.randint(12, 15)           # Exploration 
    else:
        action = np.argmax(Q[state, :])           # Exploitation 
    return action

def SARSA(env):
    
    # Initialise Q-table
    states = 50*50
    Q = np.zeros((states, 16))
    reward_count = []
    
    # For decaying epsilon greedy
    if (decay == True):
        epsilon = epsilon_decay
    else:
        epsilon = epsilon_normal
    
    # Main Loop
    for episode in range(total_episodes):
        st = env.reset()
        
        # Reshape the state so that it can be used in a 2D Q-table
        state = (st[0][0], st[0][1], st[1][0], st[1][1], st[2][0], st[2][1], st[3][0], st[3][1])
        theta1 = st[3][0]-30
        theta2 = st[3][1]-30
        state = theta1*30 + theta2
        
        #print(state)
        #if (episode % 1 == 0):
            #print("Episode {}".format(episode))
            
        step = 0                                                
        action = e_greedy(state, Q, epsilon)                    
        done = False

        # The following while loop illustrates one episode.
        while step < 50:

            ns, reward, done, _, _ = env.step(action)   # Take a step and observe results
            
            # Reshape the next state to be updated in 2D Q-table
            ntheta1 = ns[0][0]-30
            ntheta2 = ns[0][1]-30
            next_state= ntheta1*30 + ntheta2
            next_action = e_greedy(next_state, Q, epsilon)      
            
            # SARSA update rule
            predicted_value = Q[state, action]                
            target_value = reward + (gamma * Q[next_state, next_action])
            Q[state, action] += alpha * (target_value - predicted_value)
            
            state  = next_state
            action = next_action
            step   += 1 
            
            # Decaying epsilon greedy formula
            if decay == True:
                if (epsilon > min_epsilon):
                    epsilon = epsilon*(1-decay_rate)**step 
            if done:
                break
                
        reward_count.append(reward)
        print("Episode {} Average rewards = {}". format(episode+1, np.mean(reward_count)), end = '\r')
        time.sleep(0.25)
    
    return (Q, reward_count)

env = PianoHandEnv(test_key)
Q_new, reward_count = SARSA(env)

## Results obtained using SARSA

Plotting the moving average of rewards earned over all episodes.

In [ ]:
moving_average_rewards(reward_count, total_episodes, 1)